In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import gym

In [4]:
env = gym.make('CartPole-v0')

In [21]:
class cartpole:
    def __init__(self, theta, gamma, train_param):
        self.theta = theta
        self.gamma = gamma
        self.train_param = train_param
        
    def decaysum(self, rewards):
        """Calculate sum of future rewards with decay factor gamma"""
        length = rewards.shape[0]
        weights = (self.gamma * np.ones(length)) ** np.arange(length)
        weightedsum = np.dot(rewards, weights)
        return weightedsum
    
    def action_simple(self, observation):
        """Move left if pole is leaning left and right otherwise"""
        if observation[2] < 0: return 0
        else: return 1
        
    def action_sigmoid(self, observation, theta):
        """Move left or right based on a sigmoid function"""
        sigmoid = 1 / (1 + np.exp(-np.dot(observation, theta)))
        # Go right (1) with probability sigmoid, else go left (0)
        rand_cf = np.random.rand(1)
        if rand_cf < sigmoid: return 1
        else: return 0 
    
    def update_theta(self, theta, actions, states, fwd_rewards):
        """Find the direction of gradient descent to update theta"""
        num_steps = actions.shape[0]
        update = np.zeros(4)
        for step in range(num_steps):
            factor1 = int(actions[step]==0) - int(actions[step]==1)
            factor2 = 1 - 1/(1+np.exp(-np.dot(observation, theta)))
            update = update + factor1 * factor2 * states[step]
        return update
    
    def run_simulations(self, num_episodes, episode_length):
        """Run simulations and find direction in which to update theta"""
        
        theta = self.theta
        update_all = np.zeros(4)
        train_param = self.train_param
        gamma = self.gamma
        
        # Shows the number of steps until the pole falls. If the
        # pole does not fall, we want it to give the episode_length
        success_array = episode_length * np.ones(num_episodes)
        
        for episode in range(num_episodes):
            observation = env.reset()
            
            # Keep log of the states, actions, and rewards
            states = np.zeros(shape=(episode_length, 4))
            actions = np.zeros(shape=(episode_length))
            rewards = np.zeros(shape=(episode_length))
    
            # Run each episode and record relevant data
            for step in range(episode_length):
                states[step] = observation
                action = self.action_sigmoid(observation, theta)
                observation, reward, done, info = env.step(action)
                actions[step] = action
                rewards[step] = reward
                
                # Finish the loop if pole has tilted too far
                if done:
                    success_array[episode] = step + 1
                    break
                
            # Input the total (decaying) future rewards
            fwd_rewards = np.zeros(shape=(episode_length))
            for step in range(episode_length):
                fwd_rewards[step] = self.decaysum(rewards[step:])
            
            update = self.update_theta(theta, actions, states, fwd_rewards)
            update_all = update_all + update

        print(np.mean(success_array))
        print(update_all)
        print(theta + train_param * update_all)

In [60]:
theta = [0.50, 22.76, -2.16, -40.25]
gamma = 0.9
train_param = 0.001
model = cartpole(theta=theta, gamma=gamma, \
        train_param=train_param)

In [61]:
model.run_simulations(10000, 250)

9.3008
[ -2585.37254602 -51735.36879633   3741.30504894  79175.82563172]
[ -2.08537255 -28.9753688    1.58130505  38.92582563]


In [63]:
def decaysum(rewards, gamma):
    """Calculate sum of future rewards with decay factor gamma"""
    length = rewards.shape[0]
    weights = (gamma * np.ones(length)) ** np.arange(length)
    weightedsum = np.dot(rewards, weights)
    return weightedsum
        
def action_sigmoid(observation, theta):
    """Move left or right based on a sigmoid function"""
    sigmoid = 1 / (1 + np.exp(-np.dot(observation, theta)))
    # Go right (1) with probability sigmoid, else go left (0)
    rand_cf = np.random.rand(1)
    if rand_cf < sigmoid: return 1
    else: return 0 
    
def update_theta(theta, actions, states, fwd_rewards):
    """Find the direction of gradient descent to update theta"""
    num_steps = actions.shape[0]
    update = np.zeros(4)
    for step in range(num_steps):
        factor1 = int(actions[step]==0) - int(actions[step]==1)
        factor2 = 1 - 1/(1+np.exp(-np.dot(observation, theta)))
        update = update + factor1 * factor2 * states[step]
    return update

In [72]:
num_episodes = 1000
episode_length = 250
train_param = 0.001     # Training parameter
gamma = 0.9    # Decay rate

theta = np.array([0., 0., 0., 0.])
#theta = np.array([-0.8, 3.7, 0.7, 6.8])
update_all = np.zeros(4)

# Shows the number of steps until the pole falls. If the
# pole does not fall, we want it to give the episode_length
success_array = episode_length * np.ones(num_episodes)

for episode in range(num_episodes):
    observation = env.reset()
    
    # Keep a log of the states, actions rewards and future 
    # weighted decay rewards
    states = np.zeros(shape=(episode_length, 4))
    actions = np.zeros(shape=(episode_length))
    rewards = np.zeros(shape=(episode_length))
    
    # Do 100 steps, take a step and record all relevant data
    for step in range(episode_length):
        states[step] = observation
        action = action_sigmoid(observation, theta)
        actions[step] = action
        observation, reward, done, info = env.step(action)
        rewards[step] = reward
        
        # Finish the loop if the pole has tilted too far
        if done:
            success_array[episode] = step + 1
            break
            
    # Input the total (decaying) future rewards
    fwd_rewards = np.zeros(shape=(episode_length))
    for step in range(episode_length):
        fwd_rewards[step] = decaysum(rewards[step:], gamma)
        
    update = update_theta(theta, actions, states, fwd_rewards)
    update_all = update_all + update

print(np.mean(success_array))
print(update_all)
print(theta + train_param * update_all)

22.259
[  8.61513744  53.37364918 -10.21788828 -78.59133177]
[ 0.00861514  0.05337365 -0.01021789 -0.07859133]


In [ ]:
"""Results over 10000 episodes:

Action                     Mean time until done
constant (0 or 1)          9.35
action_simple              42.17

"""

In [ ]:
theta = tf.placeholder(tf.float32, shape=(4,))
state = tf.placeholder(tf.float32, shape=(4,))
dot = tf.tensordot(theta, state, axes=1)
sigmoid = tf.nn.sigmoid(dot)

In [ ]:
with tf.Session() as sess:
    feed_dict = {theta: np.array([1, 1, 1, 1]), \
                 state: np.array([1, 1, 1, 1])}
    out = sess.run(sigmoid, feed_dict=feed_dict)
    print(out)